In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import os
from transformers import TFEsmModel, EsmTokenizer

# Check available data files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


2025-09-01 13:55:50.166846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756734950.526875      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756734950.638335      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/pdb14189/PDB14189_N.txt
/kaggle/input/pdb14189/PDB14189_P.txt


In [2]:
RANDOM_SEED=229
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [3]:
def load_fasta_data(positive_file, negative_file):
    """Load protein sequences from FASTA files"""
    def parse_fasta(file_path):
        sequences = []
        current_seq = ""
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    if current_seq:
                        sequences.append(current_seq)
                    current_seq = ""
                else:
                    current_seq += line
        if current_seq:
            sequences.append(current_seq)
        return sequences
    
    positive_sequences = parse_fasta(positive_file)
    negative_sequences = parse_fasta(negative_file)
    positive_labels = [1] * len(positive_sequences)
    negative_labels = [0] * len(negative_sequences)
    all_sequences = positive_sequences + negative_sequences
    all_labels = positive_labels + negative_labels
    
    print(f"Loaded {len(positive_sequences)} positive and {len(negative_sequences)} negative sequences")
    print(f"Total sequences: {len(all_sequences)}")
    return all_sequences, all_labels

def evaluate_model(model, test_data, y_test):
    """Comprehensive model evaluation for transformer model"""
    if isinstance(test_data, list):  # For transformer model with multiple inputs
        y_pred_proba = model.predict(test_data)
    else:
        y_pred_proba = model.predict(test_data)
    
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1-Score: {f1:.4f}")
    print(f"Test AUC-ROC: {auc:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Non-DNA-binding', 'DNA-binding']))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1, 'auc': auc}

In [4]:
class PretrainedTransformerCNN:
    def __init__(self, transformer_model="facebook/esm2_t33_650M_UR50D", 
                 max_seq_len=1024, freeze_transformer=True):
        self.transformer_model = transformer_model
        self.max_seq_len = max_seq_len
        self.freeze_transformer = freeze_transformer
        
        print(f"Loading transformer model: {transformer_model}")
        self.tokenizer = EsmTokenizer.from_pretrained(transformer_model)
        self.transformer = TFEsmModel.from_pretrained(transformer_model)
        
        if freeze_transformer:
            for layer in self.transformer.layers:
                layer.trainable = False
            print("Transformer weights frozen for feature extraction")
    
    def build_model(self):
        """Build the Transformer → CNN → Classification pipeline"""
        input_ids = Input(shape=(self.max_seq_len,), dtype=tf.int32, name='input_ids')
        attention_mask = Input(shape=(self.max_seq_len,), dtype=tf.int32, name='attention_mask')
        
        # Wrap transformer call in Lambda layer
        def transformer_call(inputs):
            input_ids, attention_mask = inputs
            return self.transformer(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        
        sequence_embeddings = Lambda(transformer_call)([input_ids, attention_mask])
        
        # Multi-scale CNN
        conv_layers = []
        for kernel_size, filters in [(3, 256), (5, 256), (7, 256)]:
            conv = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu')(sequence_embeddings)
            conv = BatchNormalization()(conv)
            conv = MaxPooling1D(pool_size=2)(conv)
            conv_layers.append(conv)
        
        cnn_features = Concatenate(axis=-1)(conv_layers)
        cnn_features = Conv1D(512, 3, padding='same', activation='relu')(cnn_features)
        cnn_features = BatchNormalization()(cnn_features)
        cnn_features = Dropout(0.2)(cnn_features)
        cnn_features = Conv1D(256, 3, padding='same', activation='relu')(cnn_features)
        cnn_features = BatchNormalization()(cnn_features)
        cnn_features = GlobalMaxPooling1D()(cnn_features)
        
        # Classification head
        x = Dense(512, activation='relu')(cnn_features)
        x = Dropout(0.3)(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.1)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=[input_ids, attention_mask], outputs=outputs, name='TransformerCNN')
        return model

    
    def preprocess_sequences(self, sequences):
        """Preprocess protein sequences for the transformer"""
        spaced_sequences = [' '.join(seq) for seq in sequences]
        encoded = self.tokenizer(spaced_sequences, padding=True, truncation=True, 
                               max_length=self.max_seq_len, return_tensors="tf")
        return {'input_ids': encoded['input_ids'], 'attention_mask': encoded['attention_mask']}


In [5]:
# Load full dataset for transformer model
sequences, labels = load_fasta_data(
    positive_file='/kaggle/input/pdb14189/PDB14189_P.txt',
    negative_file='/kaggle/input/pdb14189/PDB14189_N.txt'
)

print(f"Using dataset: {len(sequences)} sequences for Transformer CNN model")
y_full = np.array(labels)
print(f"Positive samples: {np.sum(y_full)}, Negative samples: {len(y_full) - np.sum(y_full)}")

Loaded 7129 positive and 7060 negative sequences
Total sequences: 14189
Using dataset: 14189 sequences for Transformer CNN model
Positive samples: 7129, Negative samples: 7060


In [6]:
# Create transformer model
transformer_cnn = PretrainedTransformerCNN()
transformer_model = transformer_cnn.build_model()

transformer_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Preprocess full dataset
print("Preprocessing sequences...")
transformer_data = transformer_cnn.preprocess_sequences(sequences)

# Split data
indices = np.arange(len(sequences))
train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=RANDOM_SEED, stratify=y_full)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=RANDOM_SEED, stratify=y_full[temp_idx])

def safe_gather(tensor, indices):
    """Safely gather elements from tensor using numpy indices"""
    if isinstance(indices, np.ndarray):
        indices = tf.convert_to_tensor(indices, dtype=tf.int32)
    return tf.gather(tensor, indices)

# Create datasets using tf.gather
train_data = [
    safe_gather(transformer_data['input_ids'], train_idx), 
    safe_gather(transformer_data['attention_mask'], train_idx)
]
val_data = [
    safe_gather(transformer_data['input_ids'], val_idx), 
    safe_gather(transformer_data['attention_mask'], val_idx)
]
test_data = [
    safe_gather(transformer_data['input_ids'], test_idx), 
    safe_gather(transformer_data['attention_mask'], test_idx)
]

y_train_full, y_val_full, y_test_full = y_full[train_idx], y_full[val_idx], y_full[test_idx]

print(f"Training set: {len(train_idx)} samples")
print(f"Validation set: {len(val_idx)} samples") 
print(f"Test set: {len(test_idx)} samples")
# Train transformer model
transformer_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('best_transformer_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
]

Loading transformer model: facebook/esm2_t33_650M_UR50D


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

I0000 00:00:1756734993.133049      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756734993.133983      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFEsmModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'esm.embeddings.position_ids']
- This IS expected if you are initializing TFEsmModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFEsmModel from a PyTorch model that you expe

Transformer weights frozen for feature extraction
Preprocessing sequences...
Training set: 8513 samples
Validation set: 2838 samples
Test set: 2838 samples


In [ ]:
print("=== Training Transformer CNN Model ===")

transformer_history = transformer_model.fit(
    train_data, y_train_full,
    batch_size=8,
    epochs=20,
    validation_data=(val_data, y_val_full),
    callbacks=transformer_callbacks,
    verbose=1
)

=== Training Transformer CNN Model ===
Epoch 1/20


I0000 00:00:1756735101.175811     204 service.cc:148] XLA service 0x7f4a18010de0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756735101.177224     204 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1756735101.177251     204 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
W0000 00:00:1756735103.737201     204 assert_op.cc:38] Ignoring Assert operator TransformerCNN_1/lambda_1/tf_esm_model/esm/embeddings/assert_less/Assert/Assert
I0000 00:00:1756735109.193140     204 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1756735125.122523     204 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 106/1065 ━━━━━━━━━━━━━━━━━━━━ 1:01:15 4s/step - accuracy: 0.5453 - loss: 1.0421 - precision: 0.5450 - recall: 0.5687

In [ ]:
print("=== Transformer CNN Model Results ===")
transformer_results = evaluate_model(transformer_model, test_data, y_test_full)

In [ ]:
# # Save transformer results
# with open('transformer_cnn_results.pkl', 'wb') as f:
#     pickle.dump(transformer_results, f)

# print("\nTransformer results saved to 'transformer_cnn_results.pkl'")

In [ ]:
# # Load simple CNN results for comparison
# try:
#     import pickle
#     with open('simple_cnn_results.pkl', 'rb') as f:
#         simple_results = pickle.load(f)
    
#     print("\n=== Model Comparison ===")
#     print("Simple CNN+BiLSTM vs Transformer CNN")
#     print(f"Accuracy: {simple_results['accuracy']:.4f} vs {transformer_results['accuracy']:.4f}")
#     print(f"Precision: {simple_results['precision']:.4f} vs {transformer_results['precision']:.4f}")
#     print(f"Recall: {simple_results['recall']:.4f} vs {transformer_results['recall']:.4f}")
#     print(f"F1-Score: {simple_results['f1_score']:.4f} vs {transformer_results['f1_score']:.4f}")
#     print(f"AUC-ROC: {simple_results['auc']:.4f} vs {transformer_results['auc']:.4f}")
    
# except FileNotFoundError:
#     print("CNN results not found")